# <a id='toc1_'></a>[Neo4j X Python](#toc0_)



## Table of Contents<a id='toc0_'></a>    
- [Neo4j X Python](#toc1_)    
  - [Table of Contents](#toc1_1_)    
- [Using Neo4j from Python](#toc2_)    
- [Build applications with Neo4j and Python ](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Using Neo4j from Python](#toc0_)

[Using Neo4j from Python](https://neo4j.com/docs/getting-started/languages-guides/neo4j-python/)


In [1]:
!pip3 install neo4j


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')


In [11]:
from neo4j import GraphDatabase

class HelloWorldExample:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.driver.verify_connectivity()

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.execute_write(self._create_and_return_greeting, message)
            # execute_write 方法接受一个 回调函数 和 回调函数的参数
            # 确保回调函数在一个事务中执行，并在执行完后提交事务
            # 当 session.execute_write 调用回调函数时，会将事务对象 tx 作为第一个参数传递给回调函数
            print(greeting)

    @staticmethod  # 静态方法
    def _create_and_return_greeting(tx, message):
        result = tx.run(query="CREATE (a:Greeting) "
                        "SET a.message = $message "  # 设置节点的 message 属性为传入的 message 值
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]



In [12]:
hello = HelloWorldExample(uri=NEO4J_URI,
                          user=NEO4J_USERNAME,
                          password=NEO4J_PASSWORD)

hello.print_greeting("hello neo4j")
hello.close()


hello neo4j, from node 33883


# <a id='toc3_'></a>[Build applications with Neo4j and Python](https://neo4j.com/docs/python-manual/current/)  [&#8593;](#toc0_)

In [2]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')


with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    driver.verify_connectivity()

    records, summary, keys = driver.execute_query("MATCH (n) RETURN n")
    print(records)

    for record in records:
        info = record["n"]
        labels = info._labels
        print(labels)
        properties = info._properties
        print(properties)

    print("The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after,
    ))


[<Record n=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:0' labels=frozenset({'Student', 'Person'}) properties={'name': 'lzy'}>>, <Record n=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:1' labels=frozenset({'School'}) properties={'name': 'tju'}>>, <Record n=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:2' labels=frozenset({'Person'}) properties={'name': 'Alice'}>>]
frozenset({'Student', 'Person'})
{'name': 'lzy'}
frozenset({'School'})
{'name': 'tju'}
frozenset({'Person'})
{'name': 'Alice'}
The query `MATCH (n) RETURN n` returned 3 records in 29 ms.


## Write to Database

In [4]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    summary = driver.execute_query(database_="neo4j",
        query_="MERGE (:Person {name: $name})",
        name = "Alice").summary
    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))


Created 0 nodes in 33 ms.


## Read from Database

In [3]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="MATCH (p:Person) RETURN p.name AS name")

    # Loop through results and do something with them
    for record in records:
        print(record.data())  # obtain record as dict

    print("------------")

    # Summary information
    print("The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after
    ))


{'name': 'lzy'}
{'name': 'Alice'}
------------
The query `MATCH (p:Person) RETURN p.name AS name` returned 2 records in 29 ms.


## Update the Database

In [38]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    labels = "Teacher"
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_=f"MATCH (p:Person{{name:$name}}) \
            SET p:{labels} \
            RETURN p", name="Alice")
    for record in records:
        print(record.data())
        print(record)


{'p': {'name': 'Alice', 'age': '18'}}
<Record p=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:6' labels=frozenset({'Person', 'Teacher'}) properties={'name': 'Alice', 'age': '18'}>>


In [37]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="MATCH (p:Person{name:$name}) \
            SET p.age=$age \
            RETURN p", name="Alice", age="18")
    for record in records:
        print(record.data())
        print(record)


{'p': {'name': 'Alice', 'age': '18'}}
<Record p=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:6' labels=frozenset({'Person'}) properties={'name': 'Alice', 'age': '18'}>>


In [36]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="MERGE (p1:Person{name:$name1})\
            MERGE (p2:Person{name:$name2})\
            MERGE (p1)-[:Knows]->(p2)\
            RETURN p1, p2", name1="lzy", name2="Alice")
    for record in records:
        print(record.data())
        print(record)


{'p1': {'name': 'lzy'}, 'p2': {'name': 'Alice'}}
<Record p1=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:5' labels=frozenset({'Person'}) properties={'name': 'lzy'}> p2=<Node element_id='4:48dd463c-ab48-4195-a78f-cd9ee360df9b:6' labels=frozenset({'Person'}) properties={'name': 'Alice'}>>


In [35]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="MATCH (p:Person)\
            WHERE p.name=$name\
            DETACH DELETE p",
        name="alice")
    for record in records:
        print(record.data())
        print(record)


In [18]:
with GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="CALL db.propertyKeys()")
    for record in records:
        print(record.data())

    print("---")

    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="CALL db.labels()")
    for record in records:
        print(record.data())


{'propertyKey': 'name'}
---
{'label': 'Person'}
{'label': 'School'}
{'label': 'Student'}


In [34]:
with GraphDatabase.driver(uri=NEO4J_URI,
                          auth=(NEO4J_USERNAME,
                                NEO4J_PASSWORD)) as driver:
    parameters = \
    {
        "name": "Bob",
        "age": 42
    }
    records, summary, keys = driver.execute_query(
        database_="neo4j",
        query_="MERGE (:Person {name: $name})",
        parameters_=parameters,
    )

    for record in records:
        print(record.data())
